# Optimización de un mecanismo de cuatro barras utilizando Evolución Diferencial

En este notebook vamos a aplicar el algoritmo de Evolución Diferencial para resolver un problema de optimización real en el área de mecánica. Pueden encontrar información más detallada en el trabajo publicado recientemente:
https://www.mdpi.com/545532


La idea en general es sencilla, se tiene un __mecanismo__, que puede ser robótico, y la idea es __balancearlo__ de tal forma que se __minimicen el momento y la fuerza de sacudimiento__. 

Se puede ver a más detalle en el siguiente video. En la primera parte, se ve que el mecanismo NO está balanceado y se mueve un poco, incluso hasta tira la varita de madera. En la segunda parte del video, cuando el mecanismo ya está balanceado, se puede ver que ya no se mueve tanto. De hecho, el balanceo se logra agregando unos contrapesos (los tubos metálicos).

In [2]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/Bk1QmccUaZY?rel=0" frameborder="0"  allowfullscreen></iframe>

Generalmente, cuando se resuelve un problema de optimización el equipo se divide en dos partes: los expertos en el modelado del problema, para este problema la parte mecánica, y los que realizan el trabajo de optimización. En este caso, nosotros nos encargaremos de la parte de optimización, por lo que la parte mecánica la describiremos muy a grandes rasgos.

## Definición de las funciones y variables a optimizar

El problema es un mecanismo en dos dimensiones, donde la idea es agregar tres contrapesos para reducir el sacudimiento. 

Lo que se debe calcular, para cada contrapeso, es su posición (x,y) y su espesor. Por lo que una solución se puede expresar como un vector con 9 entradas:

$$ X = [x_{c1},x_{c2},x_{c3},y_{c1},y_{c2},y_{c3},m_{c1},m_{c2},m_{c3}] $$

<img src="https://www.mdpi.com/applsci/applsci-09-04115/article_deploy/html/images/applsci-09-04115-g002.png" width="500" height="500"/>

El problema de optimización está dividido en dos partes. Por un lado, se desea minimizar el momento de sacudimiento $ShM$ y por otro la fuerza de sacudimiento $ShF$. Dichas funciones están descritas a continuación y fueron proporcionadas por los expertos en mecánica.

Las funciones están diseñadas de tal forma que si su valor es igual a 1, entonces el agregar los contrapesos ni mejora ni empeora el mecanismo. Si el valor es mayor a 1, quiere decir que al agregar los contrapesos se empeora, es decir, el mecanismo tiene mayor sacudimiento. Entre más cerca esté el valor a 0, es mejor, ya que indica que el mecanismo se vuelve más estable. 

In [3]:
import numpy as np
def ShF(x_c1,x_c2,x_c3,y_c1,y_c2,y_c3,t_c1,t_c2,t_c3):
    return 2.224e-8 * np.sqrt(
    1.897e23 * t_c3 ** 2 * y_c3 ** 6 + 7.785e6 * t_c3 ** 2 * x_c3 * y_c3 ** 5 + 5.691e23 * t_c3 ** 2 * x_c3 ** 2 * y_c3 ** 4 + 1.557e7 * t_c3 ** 2 * x_c3 ** 3 * y_c3 ** 3 + 1.392e18 * t_c3 * y_c3 ** 3 + 2.742e23 * t_c2 * y_c2 ** 3 * t_c3 * y_c3 ** 3 - 1.314e22 * t_c2 * x_c2 * y_c2 ** 2 * t_c3 *
    y_c3 ** 3 + 1.025e22 * t_c2 * y_c2 ** 2 * t_c3 * y_c3 ** 3 + 2.742e23 * t_c2 * x_c2 ** 2 * y_c2 * t_c3 * y_c3 ** 3 - 1.314e22 * t_c2 * x_c2 ** 3 * t_c3 * y_c3 ** 3 + 1.025e22 * t_c2 * x_c2 ** 2 * t_c3 * y_c3 ** 3 + 3.444e22 * t_c1 * y_c1 ** 3 * t_c3 * y_c3 ** 3 + 2.562e22 * t_c1 * x_c1 * y_c1 ** 2 * t_c3 * y_c3 ** 3 + 3.444e22 *
    t_c1 * x_c1 ** 2 * y_c1 * t_c3 * y_c3 ** 3 + 2.562e22 * t_c1 * x_c1 ** 3 * t_c3 * y_c3 ** 3 + 5.691e23 * t_c3 ** 2 * x_c3 ** 4 * y_c3 ** 2 + 3.779e19 * t_c3 * x_c3 * y_c3 ** 2 + 1.314e22 * t_c2 * y_c2 ** 3 * t_c3 * x_c3 * y_c3 ** 2 + 2.742e23 * t_c2 * x_c2 * y_c2 ** 2 * t_c3 * x_c3 * y_c3 ** 2 + 1.377e22 * t_c2 * y_c2 ** 2 * t_c3 * x_c3 *
    y_c3 ** 2 + 1.314e22 * t_c2 * x_c2 ** 2 * y_c2 * t_c3 * x_c3 * y_c3 ** 2 + 2.742e23 * t_c2 * x_c2 ** 3 * t_c3 * x_c3 * y_c3 ** 2 + 1.377e22 * t_c2 * x_c2 ** 2 * t_c3 * x_c3 * y_c3 ** 2 - 2.562e22 * t_c1 * y_c1 ** 3 * t_c3 * x_c3 * y_c3 ** 2 + 3.444e22 * t_c1 * x_c1 * y_c1 ** 2 * t_c3 * x_c3 * y_c3 ** 2 - 2.562e22 * t_c1 * x_c1 ** 2 *
    y_c1 * t_c3 * x_c3 * y_c3 ** 2 + 3.444e22 * t_c1 * x_c1 ** 3 * t_c3 * x_c3 * y_c3 ** 2 + 7.785e6 * t_c3 ** 2 * x_c3 ** 5 * y_c3 + 1.392e18 * t_c3 * x_c3 ** 2 * y_c3 + 2.742e23 * t_c2 * y_c2 ** 3 * t_c3 * x_c3 ** 2 * y_c3 - 1.314e22 * t_c2 * x_c2 * y_c2 ** 2 * t_c3 * x_c3 ** 2 * y_c3 + 1.025e22 * t_c2 * y_c2 ** 2 * t_c3 * x_c3 ** 2 * y_c3
    + 2.742e23 * t_c2 * x_c2 ** 2 * y_c2 * t_c3 * x_c3 ** 2 * y_c3 - 1.314e22 * t_c2 * x_c2 ** 3 * t_c3 * x_c3 ** 2 * y_c3 + 1.025e22 * t_c2 * x_c2 ** 2 * t_c3 * x_c3 ** 2 * y_c3 + 3.444e22 * t_c1 * y_c1 ** 3 * t_c3 * x_c3 ** 2 * y_c3 + 2.562e22 * t_c1 * x_c1 * y_c1 ** 2 * t_c3 * x_c3 ** 2 * y_c3 + 3.444e22 * t_c1 * x_c1 ** 2 * y_c1 *
    t_c3 * x_c3 ** 2 * y_c3 + 2.562e22 * t_c1 * x_c1 ** 3 * t_c3 * x_c3 ** 2 * y_c3 + 1.897e23 * t_c3 ** 2 * x_c3 ** 6 + 3.779e19 * t_c3 * x_c3 ** 3 + 1.314e22 * t_c2 * y_c2 ** 3 * t_c3 * x_c3 ** 3 + 2.742e23 * t_c2 * x_c2 * y_c2 ** 2 * t_c3 * x_c3 ** 3 + 1.377e22 * t_c2 * y_c2 ** 2 * t_c3 * x_c3 ** 3 + 1.314e22 * t_c2 * x_c2 ** 2 *
    y_c2 * t_c3 * x_c3 ** 3 + 2.742e23 * t_c2 * x_c2 ** 3 * t_c3 * x_c3 ** 3 + 1.377e22 * t_c2 * x_c2 ** 2 * t_c3 * x_c3 ** 3 - 2.562e22 * t_c1 * y_c1 ** 3 * t_c3 * x_c3 ** 3 + 3.444e22 * t_c1 * x_c1 * y_c1 ** 2 * t_c3 * x_c3 ** 3 - 2.562e22 * t_c1 * x_c1 ** 2 * y_c1 * t_c3 * x_c3 ** 3 + 3.444e22 * t_c1 * x_c1 ** 3 * t_c3 * x_c3 ** 3 +
    1.115e23 * t_c2 ** 2 * y_c2 ** 6 - 8.982e6 * t_c2 ** 2 * x_c2 * y_c2 ** 5 + 7.882e21 * t_c2 ** 2 * y_c2 ** 5 + 3.346e23 * t_c2 ** 2 * x_c2 ** 2 * y_c2 ** 4 - 9.482e21 * t_c2 ** 2 * x_c2 * y_c2 ** 4 + 7.83e21 * t_c2 ** 2 * y_c2 ** 4 - 1.796e7 * t_c2 ** 2 * x_c2 ** 3 * y_c2 ** 3 + 1.576e22 * t_c2 ** 2 * x_c2 ** 2 * y_c2 ** 3 + 2.315e18 * t_c2 * y_c2 ** 3
    - 2.371e22 * t_c1 * y_c1 ** 3 * t_c2 * y_c2 ** 3 + 1.97e22 * t_c1 * x_c1 * y_c1 ** 2 * t_c2 * y_c2 ** 3 - 2.371e22 * t_c1 * x_c1 ** 2 * y_c1 * t_c2 * y_c2 ** 3 + 1.97e22 * t_c1 * x_c1 ** 3 * t_c2 * y_c2 ** 3 + 3.346e23 * t_c2 ** 2 * x_c2 ** 4 * y_c2 ** 2 - 1.896e22 * t_c2 ** 2 * x_c2 ** 3 * y_c2 ** 2 + 1.566e22 * t_c2 ** 2 * x_c2 ** 2 * y_c2 ** 2 +
    2.467e19 * t_c2 * x_c2 * y_c2 ** 2 - 1.97e22 * t_c1 * y_c1 ** 3 * t_c2 * x_c2 * y_c2 ** 2 - 2.371e22 * t_c1 * x_c1 * y_c1 ** 2 * t_c2 * x_c2 * y_c2 ** 2 - 1.97e22 * t_c1 * x_c1 ** 2 * y_c1 * t_c2 * x_c2 * y_c2 ** 2 - 2.371e22 * t_c1 * x_c1 ** 3 * t_c2 * x_c2 * y_c2 ** 2 + 3.432e18 * t_c2 * y_c2 ** 2 + 1.12e6 * t_c1 * y_c1 ** 3 *
    t_c2 * y_c2 ** 2 + 3.915e22 * t_c1 * x_c1 * y_c1 ** 2 * t_c2 * y_c2 ** 2 + 1.12e6 * t_c1 * x_c1 ** 2 * y_c1 * t_c2 * y_c2 ** 2 + 3.915e22 * t_c1 * x_c1 ** 3 * t_c2 * y_c2 ** 2 - 8.982e6 * t_c2 ** 2 * x_c2 ** 5 * y_c2 + 7.882e21 * t_c2 ** 2 * x_c2 ** 4 * y_c2 + 2.315e18 * t_c2 * x_c2 ** 2 * y_c2 - 2.371e22 * t_c1 * y_c1 ** 3 * t_c2 *
    x_c2 ** 2 * y_c2 + 1.97e22 * t_c1 * x_c1 * y_c1 ** 2 * t_c2 * x_c2 ** 2 * y_c2 - 2.371e22 * t_c1 * x_c1 ** 2 * y_c1 * t_c2 * x_c2 ** 2 * y_c2 + 1.97e22 * t_c1 * x_c1 ** 3 * t_c2 * x_c2 ** 2 * y_c2 + 1.115e23 * t_c2 ** 2 * x_c2 ** 6 - 9.482e21 * t_c2 ** 2 * x_c2 ** 5 + 7.83e21 * t_c2 ** 2 * x_c2 ** 4 + 2.467e19 * t_c2 * x_c2 ** 3 -
    1.97e22 * t_c1 * y_c1 ** 3 * t_c2 * x_c2 ** 3 - 2.371e22 * t_c1 * x_c1 * y_c1 ** 2 * t_c2 * x_c2 ** 3 - 1.97e22 * t_c1 * x_c1 ** 2 * y_c1 * t_c2 * x_c2 ** 3 - 2.371e22 * t_c1 * x_c1 ** 3 * t_c2 * x_c2 ** 3 + 3.432e18 * t_c2 * x_c2 ** 2 + 1.12e6 * t_c1 * y_c1 ** 3 * t_c2 * x_c2 ** 2 + 3.915e22 * t_c1 * x_c1 * y_c1 ** 2 * t_c2 * x_c2 ** 2 +
    1.12e6 * t_c1 * x_c1 ** 2 * y_c1 * t_c2 * x_c2 ** 2 + 3.915e22 * t_c1 * x_c1 ** 3 * t_c2 * x_c2 ** 2 + 4.894e22 * t_c1 ** 2 * y_c1 ** 6 + 7.438e6 * t_c1 ** 2 * x_c1 * y_c1 ** 5 + 1.468e23 * t_c1 ** 2 * x_c1 ** 2 * y_c1 ** 4 + 1.488e7 * t_c1 ** 2 * x_c1 ** 3 * y_c1 ** 3 - 2.425e18 * t_c1 * y_c1 ** 3 + 1.468e23 * t_c1 ** 2 * x_c1 ** 4 * y_c1 ** 2 +
    8.581e18 * t_c1 * x_c1 * y_c1 ** 2 + 7.438e6 * t_c1 ** 2 * x_c1 ** 5 * y_c1 - 2.425e18 * t_c1 * x_c1 ** 2 * y_c1 + 4.894e22 * t_c1 ** 2 * x_c1 ** 6 + 8.581e18 * t_c1 * x_c1 ** 3 + 2.022e15)

def ShM(x_c1,x_c2,x_c3,y_c1,y_c2,y_c3,t_c1,t_c2,t_c3):
    return 2.8e-53*np.sqrt(5.375e113*t_c3**2*y_c3**8-4.348e113*t_c3**2*y_c3**7+1.0e0*(2.15e114*t_c3**2*x_c3**2+2.099e113*t_c3**2*x_c3+9.452e112*t_c3**2)*y_c3**6+1.0e0*
(-1.304e114*t_c3**2*x_c3**2-6.386e112*t_c3**2*x_c3)*y_c3**5+1.0e0*(3.225e114*t_c3**2*x_c3**4+6.297e113*t_c3**2*x_c3**3+2.48e113*t_c3**2*x_c3**2+1.0e0*(8.006e113*t_c2*y_c2**4-1.029e113*t_c2*y_c2**3+1.0e0*
(1.601e114*t_c2*x_c2**2+7.592e112*t_c2*x_c2-3.816e102*t_c2)*y_c2**2-1.029e113*t_c2*x_c2**2*y_c2+8.006e113*t_c2*x_c2**4+7.592e112*t_c2*x_c2**3-3.816e102*t_c2*x_c2**2-3.578e103*t_c1*y_c1**4-
7.156e103*t_c1*x_c1**2*y_c1**2-3.578e103*t_c1*x_c1**4+4.578e109)*t_c3)*y_c3**4+1.0e0*(-1.304e114*t_c3**2*x_c3**4-1.277e113*t_c3**2*x_c3**3+1.0e0*(-3.173e113*t_c2*y_c2**4+4.913e112*t_c2*y_c2**3+
1.0e0*(-6.346e113*t_c2*x_c2**2-1.79e112*t_c2*x_c2+2.201e102*t_c2)*y_c2**2+4.913e112*t_c2*x_c2**2*y_c2-3.173e113*t_c2*x_c2**4-1.79e112*t_c2*x_c2**3+2.201e102*t_c2*x_c2**2+2.064e103*t_c1*
y_c1**4+4.127e103*t_c1*x_c1**2*y_c1**2+2.064e103*t_c1*x_c1**4-1.66e109)*t_c3)*y_c3**3+1.0e0*(2.15e114*t_c3**2*x_c3**6+6.297e113*t_c3**2*x_c3**5+2.125e113*t_c3**2*x_c3**4+1.0e0*(1.601e114*t_c2*y_c2**4-
2.057e113*t_c2*y_c2**3+1.0e0*(3.202e114*t_c2*x_c2**2+1.518e113*t_c2*x_c2-7.633e102*t_c2)*y_c2**2-2.057e113*t_c2*x_c2**2*y_c2+1.601e114*t_c2*x_c2**4+1.518e113*t_c2*x_c2**3-7.633e102*t_c2*
x_c2**2-7.156e103*t_c1*y_c1**4-1.431e104*t_c1*x_c1**2*y_c1**2-7.156e103*t_c1*x_c1**4+9.155e109)*t_c3*x_c3**2+1.0e0*(2.031e113*t_c2*y_c2**4-3.055e112*t_c2*y_c2**3+1.0e0*
(4.062e113*t_c2*x_c2**2+4.972e112*t_c2*x_c2-3.395e101*t_c2)*y_c2**2-3.055e112*t_c2*x_c2**2*y_c2+2.031e113*t_c2*x_c2**4+4.972e112*t_c2*x_c2**3-3.395e101*t_c2*x_c2**2-3.183e102*t_c1*y_c1**4-
6.366e102*t_c1*x_c1**2*y_c1**2-3.183e102*t_c1*x_c1**4+1.569e109)*t_c3*x_c3)*y_c3**2+1.0e0*(-4.348e113*t_c3**2*x_c3**6-6.386e112*t_c3**2*x_c3**5+1.0e0*(-3.173e113*t_c2*y_c2**4+4.913e112*t_c2*y_c2**3+
1.0e0*(-6.346e113*t_c2*x_c2**2-1.79e112*t_c2*x_c2+2.201e102*t_c2)*y_c2**2+4.913e112*t_c2*x_c2**2*y_c2-3.173e113*t_c2*x_c2**4-1.79e112*t_c2*x_c2**3+2.201e102*t_c2*x_c2**2+2.064e103*t_c1*
y_c1**4+4.127e103*t_c1*x_c1**2*y_c1**2+2.064e103*t_c1*x_c1**4-1.66e109)*t_c3*x_c3**2)*y_c3+5.375e113*t_c3**2*x_c3**8+2.099e113*t_c3**2*x_c3**7+5.898e112*t_c3**2*x_c3**6+1.0e0*(8.006e113*t_c2*y_c2**4-
1.029e113*t_c2*y_c2**3+1.0e0*(1.601e114*t_c2*x_c2**2+7.592e112*t_c2*x_c2-3.816e102*t_c2)*y_c2**2-1.029e113*t_c2*x_c2**2*y_c2+8.006e113*t_c2*x_c2**4+7.592e112*t_c2*x_c2**3-3.816e102*t_c2*
x_c2**2-3.578e103*t_c1*y_c1**4-7.156e103*t_c1*x_c1**2*y_c1**2-3.578e103*t_c1*x_c1**4+4.578e109)*t_c3*x_c3**4+1.0e0*(2.031e113*t_c2*y_c2**4-3.055e112*t_c2*y_c2**3+1.0e0*
(4.062e113*t_c2*x_c2**2+4.972e112*t_c2*x_c2-3.395e101*t_c2)*y_c2**2-3.055e112*t_c2*x_c2**2*y_c2+2.031e113*t_c2*x_c2**4+4.972e112*t_c2*x_c2**3-3.395e101*t_c2*x_c2**2-3.183e102*t_c1*y_c1**4-
6.366e102*t_c1*x_c1**2*y_c1**2-3.183e102*t_c1*x_c1**4+1.569e109)*t_c3*x_c3**3+3.48e113*t_c2**2*y_c2**8-9.982e112*t_c2**2*y_c2**7+1.0e0*(1.392e114*t_c2**2*x_c2**2+4.06e112*t_c2**2*x_c2+1.84e112*t_c2**2)*y_c2**6
+1.0e0*(-2.994e113*t_c2**2*x_c2**2-3.244e111*t_c2**2*x_c2+1.102e102*t_c2**2)*y_c2**5+1.0e0*(2.088e114*t_c2**2*x_c2**4+1.218e113*t_c2**2*x_c2**3+5.705e112*t_c2**2*x_c2**2+1.39e102*t_c2**2*x_c2+1.401e92*
t_c2**2+1.0e0*(-5.095e103*t_c1*y_c1**4-1.019e104*t_c1*x_c1**2*y_c1**2-5.095e103*t_c1*x_c1**4+3.737e109)*t_c2)*y_c2**4+1.0e0*
(-2.994e113*t_c2**2*x_c2**4-6.488e111*t_c2**2*x_c2**3+2.204e102*t_c2**2*x_c2**2+1.0e0*(1.033e103*t_c1*y_c1**4+2.066e103*t_c1*x_c1**2*y_c1**2+1.033e103*t_c1*x_c1**4-5.104e108)*t_c2)*y_c2**3+1.0e0*(
1.392e114*t_c2**2*x_c2**6+1.218e113*t_c2**2*x_c2**5+5.89e112*t_c2**2*x_c2**4+2.781e102*t_c2**2*x_c2**3+1.0e0*
(2.801e92*t_c2**2+1.0e0*(-1.019e104*t_c1*y_c1**4-2.038e104*t_c1*x_c1**2*y_c1**2-1.019e104*t_c1*x_c1**4+7.475e109)*t_c2)*x_c2**2+1.0e0*
(1.303e103*t_c1*y_c1**4+2.607e103*t_c1*x_c1**2*y_c1**2+1.303e103*t_c1*x_c1**4+6.72e108)*t_c2*x_c2+1.0e0*(2.626e93*t_c1*y_c1**4+5.252e93*t_c1*x_c1**2*y_c1**2+2.626e93*t_c1*x_c1**4-9.434e97)*t_c2)*
y_c2**2+1.0e0*(-9.982e112*t_c2**2*x_c2**6-3.244e111*t_c2**2*x_c2**5+1.102e102*t_c2**2*x_c2**4+1.0e0*(1.033e103*t_c1*y_c1**4+2.066e103*t_c1*x_c1**2*y_c1**2+1.033e103*t_c1*x_c1**4-5.104e108)*t_c2*x_c2**2)*
y_c2+3.48e113*t_c2**2*x_c2**8+4.06e112*t_c2**2*x_c2**7+2.025e112*t_c2**2*x_c2**6+1.39e102*t_c2**2*x_c2**5+1.0e0*
(1.401e92*t_c2**2+1.0e0*(-5.095e103*t_c1*y_c1**4-1.019e104*t_c1*x_c1**2*y_c1**2-5.095e103*t_c1*x_c1**4+3.737e109)*t_c2)*x_c2**4+1.0e0*
(1.303e103*t_c1*y_c1**4+2.607e103*t_c1*x_c1**2*y_c1**2+1.303e103*t_c1*x_c1**4+6.72e108)*t_c2*x_c2**3+1.0e0*(2.626e93*t_c1*y_c1**4+5.252e93*t_c1*x_c1**2*y_c1**2+2.626e93*t_c1*x_c1**4-9.434e97)*t_c2*
x_c2**2+1.231e94*t_c1**2*y_c1**8+4.924e94*t_c1**2*x_c1**2*y_c1**6+1.0e0*(7.386e94*t_c1**2*x_c1**4-8.845e98*t_c1)*y_c1**4+1.0e0*(4.924e94*t_c1**2*x_c1**6-1.769e99*t_c1*x_c1**2)*y_c1**2+1.231e94*t_c1**2*x_c1**8-
8.845e98*t_c1*x_c1**4+1.275e105)

Cuando se resuelve un problema de optimización, es mucho más simple enviar como parámetro a la función un sólo vector, por lo que utilizamos unas funciones que solo mandan llamar las funciones originales a optimizar. 


In [4]:
def ShM1(X):
    return ShM(X[0],X[1],X[2],X[3],X[4],X[5],X[6],X[7],X[8])
def ShF2(X):
    return ShF(X[0],X[1],X[2],X[3],X[4],X[5],X[6],X[7],X[8])


Además, como se requiere optimizar las dos funciones, pues se puede utilizar una función que sume las funciones a minimizar.

In [5]:
def f(X):
    return ShM1(X)+ShF2(X)

## Evolución diferencial

### Población inicial

El primer paso es generar la __población inicial__. Cada individuo representa una solución, y es un vector con 9 entradas.
$$ ind = [x_{c1},x_{c2},x_{c3},y_{c1},y_{c2},y_{c3},m_{c1},m_{c2},m_{c3}] $$
Los primeros tres valores, representan las posiciones en x de cada contrapeso, los siguientes tres, las posiciones en y, y finalemnte, los últimos tres valores son los espesores de los contrapesos.

Según los expertos en mecánica, los valores de las variables deben estar dentro del rango:
$$ -0.4 m <= x_{ci}, y_{ci} <= 0.4 m $$
$$ 0.005m <= m_{ci} <= 0.04 m $$

Para crear la población inicial:
$$ x_j^i = x_j^{min} + r (x_j^{max}-x_j^{min}) $$
donde $r$ es un número aleatorio entre 0 y 1.

In [7]:
def poblacion_inicial(n,bounds):
    d = len(bounds)
    poblacion = np.zeros((n, d), float)
    for i in range(d):
        poblacion[:,i] = bounds[i, 0]+ np.random.rand(n)*(bounds[i, 1]-bounds[i, 0])
    return poblacion
bounds = np.zeros((9,2),float)
bounds[:,0]=-0.4
bounds[:,1]=0.4
bounds[6:,0]=0.005
bounds[6:,1]=0.04
poblacion = poblacion_inicial(5,bounds)
#print('bounds',bounds)
print('poblacion',poblacion)

poblacion [[ 0.05480624 -0.39505886 -0.07795972 -0.3100869  -0.06917275  0.25343846
   0.03472809  0.00823019  0.00913403]
 [ 0.30532799  0.3645119   0.04036386 -0.35999504 -0.14542068 -0.26220783
   0.02215746  0.01302885  0.01241   ]
 [ 0.0465532   0.26884542 -0.36586278  0.27160726  0.35364319 -0.18541173
   0.02867524  0.01131376  0.02969001]
 [ 0.22510052 -0.25680595 -0.24697654  0.28297804 -0.22599633 -0.14608303
   0.01334987  0.01380863  0.01920678]
 [-0.3570405   0.15310957 -0.3484003  -0.24240019  0.25233494 -0.30913676
   0.00505128  0.00775018  0.02577273]]


### Fitness
Para calcular el fitness de los individuos en la población, sólo hay que mandar llamar la función a optimizar.

In [8]:
n = len(poblacion)
fitness = np.array([ f( poblacion[i,:] ) for i in range(n) ])
print(fitness)

[11.15568917 27.35320924 38.6208169  17.29756173 58.68431397]


In [ ]:
def buscar_elite(poblacion,fitness):
    # code
    return elite,eliteFitness

### Proceso evolutivo

__Mutación__

La mutación se aplica a cada uno de los individuos. El primer paso consiste en seleccionar aleatoriamente tres individuos de la población: $x^{r_1}$, $x^{r_2}$ y $x^{r_3}$. Después, el vector mutado $v^i$, se calcula como:
$$ v^i = x^{r_1} + F (x^{r_2}-x^{r_3})$$
donde $F$ es un número aleatorio entre 0 y 2.

In [ ]:
def mutacion(poblacion):
    # code
    return vpopulation

__Cruza__

La cruza combina al individuo $x^i$ con su correspondiente vector mutado $v^i$. Para lo cual se utiliza la siguiente ecuación:
$$ u^i_j = \left\{ \begin{matrix}  v_j^i & si \quad r<CR \quad or \quad j=l \\  x_j^i & de \quad otra \quad forma \end{matrix} \right. $$

donde $r$ es un número aleatorio entre 0 y 1, $CR$ es parámetro que sirve para ajustar el porcentaje de elementos que se toman del vector mutado, y finalmente, $l$ es un valor aleatorio que puede tomar los valores del conjunto $\{1,...,n\}$.

In [ ]:
def cruza(poblacion,pobv,Cr):
    # code
    return upopulation

__Selección__

En la selección, n torneo uno a uno es aplicado a $x^i$ y $u^i$, donde el mejor individuo de los dos pasa a la siguiente generación.  

In [ ]:
def seleccion(poblacion,fitness,pobu,fitu):
    # code
    return population,fitness

__Algoritmo de evolución diferencial__

In [ ]:
def evolucion_diferencial(f,bounds):    
    # code
    return elite
    
sol = evolucion_diferencial(f,bounds)
print('Solución:',sol)
